In [77]:
import numpy as np
import numpy.random as random
import math
import time
import pickle
import os
import glob
from multiprocessing import Pool
pool_n = 10
data_dir = 'data'

In [78]:
train_file_num = 100
test_file_num = 100
N = 100000

In [79]:
def MC(vol,K,T,N) :
    
    S0 = 1.
    W = random.randn(N)*np.sqrt(T)
    S = S0*np.exp( -0.5*vol**2*T + vol*W )
        
    p = np.maximum(S-K,0.)
    payoff = np.maximum(S0-K,0.)
    tv = np.mean(p)-payoff
    return tv

In [80]:
def make_data_impl(arg) :
    
    rank,data_num,N = arg 
    
    ns = time.time_ns()
    seed = int(time.time())+3*rank**2+5*rank
    random.seed(seed)
         
    vol = np.random.uniform(0.01,1.,data_num)
    T = np.random.uniform(0.01,1.,data_num)
    K0 = np.random.uniform(-2.,2.,data_num)
    K = K0*np.sqrt(T)
    K = np.exp(K)

    tv = []
    for i,(vol_,K_,T_) in enumerate(zip(vol,K,T)) :
        tv.append( MC(vol_,K_,T_,N) )
    
    x = np.vstack([vol,K0,T]).T
    y = np.array(tv).reshape(-1,1)
    return (x,y)

def make_data(data_num,N) :
    with Pool(pool_n) as p:  
        results = p.map(make_data_impl,[(rank,data_num//pool_n,N) for rank in range(pool_n)]) 
    x = [e[0] for e in results]; x = np.array(x).reshape(-1,3)
    y = [e[1] for e in results]; y = np.array(y).reshape(-1,1)
    return (x,y)    

In [81]:
%%time
for mode,file_num in zip(['train','test'],[train_file_num,test_file_num]) :
    
    print(mode)
    if not os.path.exists(os.path.join(data_dir,mode)) :
        os.mkdir(os.path.join(data_dir,mode))
    filelist = glob.glob(f'{data_dir}/{mode}/bs_{mode}_*.pkl')
    start = len(filelist)

    for i in range(start,file_num) :
        print(i,end=' ')
        x,y = make_data(10000,N)
        with open(f'{data_dir}/{mode}/bs_{mode}_{N:.0e}_{i:05d}.pkl','wb') as f: 
            pickle.dump([x,y],f)
    print('')
    print('-'*50)

train
10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 
--------------------------------------------------
test
10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 
--------------------------------------------------
CPU times: user 1.66 s, sys: 4.73 s, total: 6.39 s
Wall time: 8min 17s
